# Fedbiomed Researcher base example with OPACUS

Use for developing (autoreloads changes made across packages)

In this notebook we show how `opacus` (https://opacus.ai/) can be used in Fed-BioMed. Opacus is a library which allows to train PyTorch models with differential privacy. We will train the basic MNIST example using two nodes.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from fedbiomed.researcher.requests import Requests
req = Requests()
req.list(verbose=True)

2022-01-21 09:03:08,447 fedbiomed INFO - Component environment:
2022-01-21 09:03:08,449 fedbiomed INFO - - type = ComponentType.RESEARCHER
2022-01-21 09:03:09,336 fedbiomed INFO - Messaging researcher_bc5777a3-5027-465f-8742-3d3aa04a29c9 successfully connected to the message broker, object = <fedbiomed.common.messaging.Messaging object at 0x106e86a30>
2022-01-21 09:03:09,381 fedbiomed INFO - Listing available datasets in all nodes... 
2022-01-21 09:03:09,413 fedbiomed INFO - log from: node_edb44109-8e5f-4741-adfb-5e68136b3bab / DEBUG - Message received: {'researcher_id': 'researcher_bc5777a3-5027-465f-8742-3d3aa04a29c9', 'command': 'list'}
2022-01-21 09:03:09,419 fedbiomed INFO - log from: node_8dce5575-7403-48f0-91b6-e07e58ae5c47 / DEBUG - Message received: {'researcher_id': 'researcher_bc5777a3-5027-465f-8742-3d3aa04a29c9', 'command': 'list'}
2022-01-21 09:03:19,397 fedbiomed INFO - 
 Node: node_edb44109-8e5f-4741-adfb-5e68136b3bab | Number of Datasets: 1 
+--------+-------------+---

{'node_edb44109-8e5f-4741-adfb-5e68136b3bab': [{'name': 'MNIST',
   'data_type': 'default',
   'tags': ['#MNIST', '#dataset'],
   'description': '50.0 percent of MNIST database',
   'shape': [30000, 1, 28, 28]}],
 'node_8dce5575-7403-48f0-91b6-e07e58ae5c47': [{'name': 'MNIST',
   'data_type': 'default',
   'tags': ['#MNIST', '#dataset'],
   'description': '50.0 percent of MNIST database',
   'shape': [30000, 1, 28, 28]}]}

## Start the network
Before running this notebook, start the network with `./scripts/fedbiomed_run network`

## Setting the nodes up
It is necessary to previously configure a node:
1. `./scripts/fedbiomed_run node add`
  * Select option 2 (default)
  * Write MNIST to add MNIST to the node through `torchvision.datasets.MNIST`
  * Select the desired ratio of the MNIST dataset to be added to the current node
  * Confirm default tags by hitting "y" and ENTER
  * Pick the folder where MNIST is downloaded (this is due torch issue https://github.com/pytorch/vision/issues/3549)
  * Data must have been added (if you get a warning saying that data must be unique is because it's been already added)
  
2. Check that your data has been added by executing `./scripts/fedbiomed_run node list`
3. Run the node using `./scripts/fedbiomed_run node run`. Wait until you get `Starting task manager`. it means you are online.

3. Following the same procedure, create another node with MNIST.

## Define an experiment model and parameters

Declare a torch.nn MyTrainingPlan class to send for training on the node

In [3]:
from fedbiomed.researcher.environ import environ
import tempfile
tmp_dir_model = tempfile.TemporaryDirectory(dir=environ['TMP_DIR']+'/')
model_file = tmp_dir_model.name + '/class_export_mnist.py'

In the cell below, we are going to define the model using opacus for differential privacy. For this example, we are going to use the function `make_private` from `opacus.privacy_engine`. Two hyperparameters should be defined:
* `noise_multiplier`: The ratio of the standard deviation of the Gaussian noise to the L2-sensitivity of the function to which the noise is added (How much noise to add)
* `max_grad_norm`: The maximum norm of the per-sample gradients. Any gradient with norm higher than this will be clipped to this value.

It is worth noting that in order to use the opacus `PrivacyEngine` class we need to properly define as training plan attributes a `model`, a `dataloader` and an `optimizer`.

In [4]:
%%writefile "$model_file"

import torch
import torch.nn as nn
from fedbiomed.common.torchnnDP import TorchTrainingDPPlan
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

# Here we define the model to be used. 
# You can use any class name (here 'Net')
class MyTrainingPlan(TorchTrainingDPPlan):
    def __init__(self, model_args):
        super(MyTrainingPlan, self).__init__()
        
        # Here we define the custom dependencies that will be needed by our custom Dataloader
        # In this case, we need the torch DataLoader classes
        # Since we will train on MNIST, we need datasets and transform from torchvision
        deps = ["from torchvision import datasets, transforms",
                "from torch.utils.data import DataLoader",]
        self.add_dependency(deps)
        
        self.diff_privacy = model_args['diff_privacy']
        self.privacy_func = model_args['privacy_func']
        self.noise_multiplier = model_args['noise_multiplier']
        self.max_grad_norm = model_args['max_grad_norm']
        
        self.model = self.make_model()
        self.optimizer = torch.optim.Adam(self.model.parameters(), lr=lr)
        
    def make_model(self):
        model = nn.Sequential(nn.Conv2d(1, 32, 3, 1),
                              nn.ReLU(),
                              nn.Conv2d(32, 64, 3, 1),
                              nn.ReLU(),
                              nn.MaxPool2d(2),
                              nn.Dropout(0.25),
                              nn.Flatten(),
                              nn.Linear(9216, 128),
                              nn.ReLU(),
                              nn.Dropout(0.5),
                              nn.Linear(128, 10),
                              nn.LogSoftmax(dim=1))
        return model

    def forward(self, x):
        return self.model(x)

    def training_data(self, batch_size = 48):
        # Custom torch Dataloader for MNIST data
        transform = transforms.Compose([transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,))])
        dataset1 = datasets.MNIST(self.dataset_path, train=True, download=False, transform=transform)
        train_kwargs = {'batch_size': batch_size, 'shuffle': True}
        data_loader = torch.utils.data.DataLoader(dataset1, **train_kwargs)
        return data_loader
    
    def training_step(self, data, target):
        output = self.forward(data)
        loss   = torch.nn.functional.nll_loss(output, target)
        return loss

Writing /Users/balelli/ownCloud/INRIA_EPIONE/FedBioMed/fedbiomed/var/tmp/tmpwrt0_h02/class_export_mnist.py


This group of arguments correspond respectively:
* `model_args`: a dictionary with the arguments related to the model (e.g. number of layers, features, etc.). This will be passed to the model class on the node side. For instance, the privacy parameters should be passed here.
* `training_args`: a dictionary containing the arguments for the training routine (e.g. batch size, learning rate, epochs, etc.). This will be passed to the routine on the node side.

**NOTE:** typos and/or lack of positional (required) arguments will raise error. 🤓

In [5]:
model_args = {'diff_privacy': True, 'privacy_func': 'make_private', 'noise_multiplier':1., 'max_grad_norm':1.0}

training_args = {
    'batch_size': 48, 
    'lr': 1e-3, 
    'epochs': 3, 
    'dry_run': False,  
    'batch_maxnum': 250 # Fast pass for development : only use ( batch_maxnum * batch_size ) samples
}

## Declare and run the experiment

- search nodes serving data for these `tags`, optionally filter on a list of node ID with `nodes`
- run a round of local training on nodes with model defined in `model_path` + federation with `aggregator`
- run for `rounds` rounds, applying the `node_selection_strategy` between the rounds

In [6]:
from fedbiomed.researcher.experiment import Experiment
from fedbiomed.researcher.aggregators.fedavg import FedAverage

tags =  ['#MNIST', '#dataset']
rounds = 3

exp = Experiment(tags=tags,
                 #nodes=None,
                 model_path=model_file,
                 model_args=model_args,
                 model_class='MyTrainingPlan',
                 training_args=training_args,
                 rounds=rounds,
                 aggregator=FedAverage(),
                 node_selection_strategy=None)

2022-01-21 09:04:07,119 fedbiomed INFO - Searching dataset with data tags: ['#MNIST', '#dataset'] for all nodes
2022-01-21 09:04:07,134 fedbiomed INFO - log from: node_8dce5575-7403-48f0-91b6-e07e58ae5c47 / DEBUG - Message received: {'researcher_id': 'researcher_bc5777a3-5027-465f-8742-3d3aa04a29c9', 'tags': ['#MNIST', '#dataset'], 'command': 'search'}
2022-01-21 09:04:07,140 fedbiomed INFO - log from: node_edb44109-8e5f-4741-adfb-5e68136b3bab / DEBUG - Message received: {'researcher_id': 'researcher_bc5777a3-5027-465f-8742-3d3aa04a29c9', 'tags': ['#MNIST', '#dataset'], 'command': 'search'}
2022-01-21 09:04:17,130 fedbiomed INFO - Node selected for training -> node_8dce5575-7403-48f0-91b6-e07e58ae5c47
2022-01-21 09:04:17,136 fedbiomed INFO - Node selected for training -> node_edb44109-8e5f-4741-adfb-5e68136b3bab
2022-01-21 09:04:17,143 fedbiomed INFO - Checking data quality of federated datasets...
2022-01-21 09:04:17,852 fedbiomed DEBUG - torchnn saved model filename: /Users/balelli/o

Let's start the experiment.

By default, this function doesn't stop until all the `rounds` are done for all the nodes

In [7]:
exp.run()

2022-01-21 09:04:27,259 fedbiomed INFO - Sampled nodes in round 0 ['node_8dce5575-7403-48f0-91b6-e07e58ae5c47', 'node_edb44109-8e5f-4741-adfb-5e68136b3bab']
01/21/2022 09:04:27:INFO:Sampled nodes in round 0 ['node_8dce5575-7403-48f0-91b6-e07e58ae5c47', 'node_edb44109-8e5f-4741-adfb-5e68136b3bab']
2022-01-21 09:04:27,261 fedbiomed INFO - Send message to node node_8dce5575-7403-48f0-91b6-e07e58ae5c47 - {'researcher_id': 'researcher_bc5777a3-5027-465f-8742-3d3aa04a29c9', 'job_id': 'f899a968-6dad-4fe9-9f87-778158c56298', 'training_args': {'batch_size': 48, 'lr': 0.001, 'epochs': 3, 'dry_run': False, 'batch_maxnum': 250}, 'model_args': {'diff_privacy': True, 'privacy_func': 'make_private', 'noise_multiplier': 1.0, 'max_grad_norm': 1.0}, 'command': 'train', 'model_url': 'http://localhost:8844/media/uploads/2022/01/21/my_model_aa71d89a-5986-4efd-8b3c-7a8d190a7e2f.py', 'params_url': 'http://localhost:8844/media/uploads/2022/01/21/aggregated_params_init_d1333807-f435-49a1-8653-1910fb64e68c.pt',

01/21/2022 09:04:27:INFO:log from: node_edb44109-8e5f-4741-adfb-5e68136b3bab / DEBUG - Message received: {'researcher_id': 'researcher_bc5777a3-5027-465f-8742-3d3aa04a29c9', 'job_id': 'f899a968-6dad-4fe9-9f87-778158c56298', 'training_args': {'batch_size': 48, 'lr': 0.001, 'epochs': 3, 'dry_run': False, 'batch_maxnum': 250}, 'model_args': {'diff_privacy': True, 'privacy_func': 'make_private', 'noise_multiplier': 1.0, 'max_grad_norm': 1.0}, 'command': 'train', 'model_url': 'http://localhost:8844/media/uploads/2022/01/21/my_model_aa71d89a-5986-4efd-8b3c-7a8d190a7e2f.py', 'params_url': 'http://localhost:8844/media/uploads/2022/01/21/aggregated_params_init_d1333807-f435-49a1-8653-1910fb64e68c.pt', 'model_class': 'MyTrainingPlan', 'training_data': {'node_edb44109-8e5f-4741-adfb-5e68136b3bab': ['dataset_7f491de7-9375-4f96-a07d-f58bca25deb5']}}
2022-01-21 09:04:27,357 fedbiomed INFO - log from: node_edb44109-8e5f-4741-adfb-5e68136b3bab / DEBUG - [TASKS QUEUE] Item:{'researcher_id': 'researcher

01/21/2022 09:05:32:INFO:log from: node_edb44109-8e5f-4741-adfb-5e68136b3bab / INFO - Train Epoch: 1 [3710/60000 (6%)]	Loss: 2.203482
2022-01-21 09:05:34,767 fedbiomed INFO - log from: node_8dce5575-7403-48f0-91b6-e07e58ae5c47 / INFO - Train Epoch: 1 [3640/60000 (6%)]	Loss: 2.161693
01/21/2022 09:05:34:INFO:log from: node_8dce5575-7403-48f0-91b6-e07e58ae5c47 / INFO - Train Epoch: 1 [3640/60000 (6%)]	Loss: 2.161693
2022-01-21 09:05:42,332 fedbiomed INFO - log from: node_edb44109-8e5f-4741-adfb-5e68136b3bab / INFO - Train Epoch: 1 [4320/60000 (6%)]	Loss: 2.183929
01/21/2022 09:05:42:INFO:log from: node_edb44109-8e5f-4741-adfb-5e68136b3bab / INFO - Train Epoch: 1 [4320/60000 (6%)]	Loss: 2.183929
2022-01-21 09:05:44,619 fedbiomed INFO - log from: node_8dce5575-7403-48f0-91b6-e07e58ae5c47 / INFO - Train Epoch: 1 [4560/60000 (6%)]	Loss: 2.165349
01/21/2022 09:05:44:INFO:log from: node_8dce5575-7403-48f0-91b6-e07e58ae5c47 / INFO - Train Epoch: 1 [4560/60000 (6%)]	Loss: 2.165349
2022-01-21 09:

01/21/2022 09:07:24:INFO:log from: node_8dce5575-7403-48f0-91b6-e07e58ae5c47 / INFO - Train Epoch: 1 [10500/60000 (17%)]	Loss: 1.528210
2022-01-21 09:07:27,799 fedbiomed INFO - log from: node_edb44109-8e5f-4741-adfb-5e68136b3bab / INFO - Train Epoch: 1 [11220/60000 (18%)]	Loss: 1.558349
01/21/2022 09:07:27:INFO:log from: node_edb44109-8e5f-4741-adfb-5e68136b3bab / INFO - Train Epoch: 1 [11220/60000 (18%)]	Loss: 1.558349
2022-01-21 09:07:32,486 fedbiomed INFO - log from: node_8dce5575-7403-48f0-91b6-e07e58ae5c47 / INFO - Train Epoch: 1 [8360/60000 (18%)]	Loss: 1.397049
01/21/2022 09:07:32:INFO:log from: node_8dce5575-7403-48f0-91b6-e07e58ae5c47 / INFO - Train Epoch: 1 [8360/60000 (18%)]	Loss: 1.397049
2022-01-21 09:07:41,322 fedbiomed INFO - log from: node_edb44109-8e5f-4741-adfb-5e68136b3bab / INFO - Train Epoch: 1 [11270/60000 (18%)]	Loss: 1.529714
01/21/2022 09:07:41:INFO:log from: node_edb44109-8e5f-4741-adfb-5e68136b3bab / INFO - Train Epoch: 1 [11270/60000 (18%)]	Loss: 1.529714
20

01/21/2022 09:09:16:INFO:log from: node_8dce5575-7403-48f0-91b6-e07e58ae5c47 / INFO - Train Epoch: 2 [5000/60000 (8%)]	Loss: 1.480561
2022-01-21 09:09:18,355 fedbiomed INFO - log from: node_edb44109-8e5f-4741-adfb-5e68136b3bab / INFO - Train Epoch: 2 [4700/60000 (8%)]	Loss: 1.774979
01/21/2022 09:09:18:INFO:log from: node_edb44109-8e5f-4741-adfb-5e68136b3bab / INFO - Train Epoch: 2 [4700/60000 (8%)]	Loss: 1.774979
2022-01-21 09:09:24,457 fedbiomed INFO - log from: node_8dce5575-7403-48f0-91b6-e07e58ae5c47 / INFO - Train Epoch: 2 [5830/60000 (9%)]	Loss: 1.333541
01/21/2022 09:09:24:INFO:log from: node_8dce5575-7403-48f0-91b6-e07e58ae5c47 / INFO - Train Epoch: 2 [5830/60000 (9%)]	Loss: 1.333541
2022-01-21 09:09:25,603 fedbiomed INFO - log from: node_edb44109-8e5f-4741-adfb-5e68136b3bab / INFO - Train Epoch: 2 [3740/60000 (9%)]	Loss: 0.860678
01/21/2022 09:09:25:INFO:log from: node_edb44109-8e5f-4741-adfb-5e68136b3bab / INFO - Train Epoch: 2 [3740/60000 (9%)]	Loss: 0.860678
2022-01-21 09:

...

01/21/2022 09:26:02:INFO:log from: node_8dce5575-7403-48f0-91b6-e07e58ae5c47 / INFO - Train Epoch: 1 [7560/60000 (14%)]	Loss: 1.321575
2022-01-21 09:26:05,683 fedbiomed INFO - log from: node_edb44109-8e5f-4741-adfb-5e68136b3bab / INFO - Train Epoch: 1 [8740/60000 (15%)]	Loss: 1.620139
01/21/2022 09:26:05:INFO:log from: node_edb44109-8e5f-4741-adfb-5e68136b3bab / INFO - Train Epoch: 1 [8740/60000 (15%)]	Loss: 1.620139
2022-01-21 09:26:08,492 fedbiomed INFO - log from: node_8dce5575-7403-48f0-91b6-e07e58ae5c47 / INFO - Train Epoch: 1 [9120/60000 (15%)]	Loss: 1.527936
01/21/2022 09:26:08:INFO:log from: node_8dce5575-7403-48f0-91b6-e07e58ae5c47 / INFO - Train Epoch: 1 [9120/60000 (15%)]	Loss: 1.527936
2022-01-21 09:26:12,000 fedbiomed INFO - log from: node_edb44109-8e5f-4741-adfb-5e68136b3bab / INFO - Train Epoch: 1 [9000/60000 (16%)]	Loss: 1.196599
01/21/2022 09:26:12:INFO:log from: node_edb44109-8e5f-4741-adfb-5e68136b3bab / INFO - Train Epoch: 1 [9000/60000 (16%)]	Loss: 1.196599
2022-01

01/21/2022 09:27:35:INFO:log from: node_edb44109-8e5f-4741-adfb-5e68136b3bab / INFO - Train Epoch: 2 [3290/60000 (6%)]	Loss: 0.865019
2022-01-21 09:27:41,394 fedbiomed INFO - log from: node_8dce5575-7403-48f0-91b6-e07e58ae5c47 / INFO - Train Epoch: 2 [3640/60000 (6%)]	Loss: 0.921556
01/21/2022 09:27:41:INFO:log from: node_8dce5575-7403-48f0-91b6-e07e58ae5c47 / INFO - Train Epoch: 2 [3640/60000 (6%)]	Loss: 0.921556
2022-01-21 09:27:47,253 fedbiomed INFO - log from: node_edb44109-8e5f-4741-adfb-5e68136b3bab / INFO - Train Epoch: 2 [4480/60000 (6%)]	Loss: 1.489730
01/21/2022 09:27:47:INFO:log from: node_edb44109-8e5f-4741-adfb-5e68136b3bab / INFO - Train Epoch: 2 [4480/60000 (6%)]	Loss: 1.489730
2022-01-21 09:27:51,983 fedbiomed INFO - log from: node_8dce5575-7403-48f0-91b6-e07e58ae5c47 / INFO - Train Epoch: 2 [4880/60000 (6%)]	Loss: 1.227708
01/21/2022 09:27:51:INFO:log from: node_8dce5575-7403-48f0-91b6-e07e58ae5c47 / INFO - Train Epoch: 2 [4880/60000 (6%)]	Loss: 1.227708
2022-01-21 09:

01/21/2022 09:29:30:INFO:log from: node_8dce5575-7403-48f0-91b6-e07e58ae5c47 / INFO - Train Epoch: 2 [10920/60000 (17%)]	Loss: 1.253208
2022-01-21 09:29:34,245 fedbiomed INFO - log from: node_edb44109-8e5f-4741-adfb-5e68136b3bab / INFO - Train Epoch: 2 [11220/60000 (18%)]	Loss: 0.840325
01/21/2022 09:29:34:INFO:log from: node_edb44109-8e5f-4741-adfb-5e68136b3bab / INFO - Train Epoch: 2 [11220/60000 (18%)]	Loss: 0.840325
2022-01-21 09:29:40,285 fedbiomed INFO - log from: node_8dce5575-7403-48f0-91b6-e07e58ae5c47 / INFO - Train Epoch: 2 [11880/60000 (18%)]	Loss: 1.163492
01/21/2022 09:29:40:INFO:log from: node_8dce5575-7403-48f0-91b6-e07e58ae5c47 / INFO - Train Epoch: 2 [11880/60000 (18%)]	Loss: 1.163492
2022-01-21 09:29:45,245 fedbiomed INFO - log from: node_edb44109-8e5f-4741-adfb-5e68136b3bab / INFO - Train Epoch: 2 [7360/60000 (18%)]	Loss: 1.238004
01/21/2022 09:29:45:INFO:log from: node_edb44109-8e5f-4741-adfb-5e68136b3bab / INFO - Train Epoch: 2 [7360/60000 (18%)]	Loss: 1.238004
20

01/21/2022 09:31:11:INFO:log from: node_edb44109-8e5f-4741-adfb-5e68136b3bab / INFO - Train Epoch: 3 [5700/60000 (8%)]	Loss: 0.979775
2022-01-21 09:31:15,544 fedbiomed INFO - log from: node_8dce5575-7403-48f0-91b6-e07e58ae5c47 / INFO - Train Epoch: 3 [5400/60000 (8%)]	Loss: 1.477131
01/21/2022 09:31:15:INFO:log from: node_8dce5575-7403-48f0-91b6-e07e58ae5c47 / INFO - Train Epoch: 3 [5400/60000 (8%)]	Loss: 1.477131
2022-01-21 09:31:18,010 fedbiomed INFO - log from: node_edb44109-8e5f-4741-adfb-5e68136b3bab / INFO - Train Epoch: 3 [4620/60000 (9%)]	Loss: 1.734594
01/21/2022 09:31:18:INFO:log from: node_edb44109-8e5f-4741-adfb-5e68136b3bab / INFO - Train Epoch: 3 [4620/60000 (9%)]	Loss: 1.734594
2022-01-21 09:31:23,482 fedbiomed INFO - log from: node_8dce5575-7403-48f0-91b6-e07e58ae5c47 / INFO - Train Epoch: 3 [6930/60000 (9%)]	Loss: 1.219931
01/21/2022 09:31:23:INFO:log from: node_8dce5575-7403-48f0-91b6-e07e58ae5c47 / INFO - Train Epoch: 3 [6930/60000 (9%)]	Loss: 1.219931
2022-01-21 09:

01/21/2022 09:33:03:INFO:log from: node_8dce5575-7403-48f0-91b6-e07e58ae5c47 / INFO - Train Epoch: 3 [10080/60000 (19%)]	Loss: 0.608435
2022-01-21 09:33:06,682 fedbiomed INFO - log from: node_edb44109-8e5f-4741-adfb-5e68136b3bab / DEBUG - Reached 250 batches for this epoch, ignore remaining data
01/21/2022 09:33:06:INFO:log from: node_edb44109-8e5f-4741-adfb-5e68136b3bab / DEBUG - Reached 250 batches for this epoch, ignore remaining data
2022-01-21 09:33:07,819 fedbiomed INFO - log from: node_edb44109-8e5f-4741-adfb-5e68136b3bab / INFO - results uploaded successfully 
01/21/2022 09:33:07:INFO:log from: node_edb44109-8e5f-4741-adfb-5e68136b3bab / INFO - results uploaded successfully 
2022-01-21 09:33:09,978 fedbiomed INFO - log from: node_8dce5575-7403-48f0-91b6-e07e58ae5c47 / DEBUG - Reached 250 batches for this epoch, ignore remaining data
01/21/2022 09:33:09:INFO:log from: node_8dce5575-7403-48f0-91b6-e07e58ae5c47 / DEBUG - Reached 250 batches for this epoch, ignore remaining data
20

Local training results for each round and each node are available in `exp.training_replies` (index 0 to (`rounds` - 1) ).

For example you can view the training results for the last round below.

Different timings (in seconds) are reported for each dataset of a node participating in a round :
- `rtime_training` real time (clock time) spent in the training function on the node
- `ptime_training` process time (user and system CPU) spent in the training function on the node
- `rtime_total` real time (clock time) spent in the researcher between sending the request and handling the response, at the `Job()` layer

In [8]:
print("\nList the training rounds : ", exp.training_replies.keys())

print("\nList the nodes for the last training round and their timings : ")
round_data = exp.training_replies[rounds - 1].data
for c in range(len(round_data)):
    print("\t- {id} :\
    \n\t\trtime_training={rtraining:.2f} seconds\
    \n\t\tptime_training={ptraining:.2f} seconds\
    \n\t\trtime_total={rtotal:.2f} seconds".format(id = round_data[c]['node_id'],
        rtraining = round_data[c]['timing']['rtime_training'],
        ptraining = round_data[c]['timing']['ptime_training'],
        rtotal = round_data[c]['timing']['rtime_total']))
print('\n')
    
exp.training_replies[rounds - 1].dataframe


List the training rounds :  dict_keys([0, 1, 2])

List the nodes for the last training round and their timings : 
	- node_edb44109-8e5f-4741-adfb-5e68136b3bab :    
		rtime_training=560.13 seconds    
		ptime_training=675.14 seconds    
		rtime_total=570.46 seconds
	- node_8dce5575-7403-48f0-91b6-e07e58ae5c47 :    
		rtime_training=563.43 seconds    
		ptime_training=680.31 seconds    
		rtime_total=570.74 seconds




,success,msg,dataset_id,node_id,params_path,params,timing
0,True,,dataset_7f491de7-9375-4f96-a07d-f58bca25deb5,node_edb44109-8e5f-4741-adfb-5e68136b3bab,/Users/balelli/ownCloud/INRIA_EPIONE/FedBioMed...,"{'model._module.0.weight': [[tensor([[-0.3079,...","{'rtime_training': 560.1312185659999, 'ptime_t..."
1,True,,dataset_7adabeb5-d317-4167-a9e5-71b3cd07ab42,node_8dce5575-7403-48f0-91b6-e07e58ae5c47,/Users/balelli/ownCloud/INRIA_EPIONE/FedBioMed...,"{'model._module.0.weight': [[tensor([[-0.3063,...","{'rtime_training': 563.4319361919997, 'ptime_t..."


Federated parameters for each round are available in `exp.aggregated_params` (index 0 to (`rounds` - 1) ).

For example you can view the federated parameters for the last round of the experiment :

In [9]:
print("\nList the training rounds : ", exp.aggregated_params.keys())

print("\nAccess the federated params for the last training round :")
print("\t- params_path: ", exp.aggregated_params[rounds - 1]['params_path'])
print("\t- parameter data: ", exp.aggregated_params[rounds - 1]['params'].keys())



List the training rounds :  dict_keys([0, 1, 2])

Access the federated params for the last training round :
	- params_path:  /Users/balelli/ownCloud/INRIA_EPIONE/FedBioMed/fedbiomed/var/experiments/Experiment_0002/aggregated_params_13813343-e815-493f-9130-b5d3ee09521d.pt
	- parameter data:  odict_keys(['model.0.weight', 'model.0.bias', 'model.2.weight', 'model.2.bias', 'model.7.weight', 'model.7.bias', 'model.10.weight', 'model.10.bias'])


# Testing

We define a little testing routine to extract the accuracy metrics on the testing dataset

In [10]:
import torch
import torch.nn.functional as F


def testing_Accuracy(model, data_loader):
    model.eval()
    test_loss = 0
    correct = 0
    device = 'cpu'

    correct = 0
    
    with torch.no_grad():
        for data, target in data_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

        pred = output.argmax(dim=1, keepdim=True)

    test_loss /= len(data_loader.dataset)
    accuracy = 100* correct/len(data_loader.dataset)

    return(test_loss, accuracy)

In [11]:
from torchvision import datasets, transforms
import os

local_mnist = os.path.join(environ['TMP_DIR'], 'local_mnist')

transform = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize((0.1307,), (0.3081,))
        ])

test_set = datasets.MNIST(root = local_mnist, download = True, train = False, transform = transform)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=64, shuffle=True)

fed_model = exp.model_instance
fed_model.load_state_dict(exp.aggregated_params[rounds - 1]['params'])

acc_federated = testing_Accuracy(fed_model, test_loader)

print('\nAccuracy federated training:  {:.4f}'.format(acc_federated[1]))

print('\nError federated training:  {:.4f}'.format(acc_federated[0]))


Accuracy federated training:  86.4100

Error federated training:  0.5747


2022-01-21 09:43:05,118 fedbiomed INFO - log from: node_edb44109-8e5f-4741-adfb-5e68136b3bab / DEBUG - Message received: {'researcher_id': 'researcher_bc5777a3-5027-465f-8742-3d3aa04a29c9', 'tags': ['#MNIST', '#dataset'], 'command': 'search'}
01/21/2022 09:43:05:INFO:log from: node_edb44109-8e5f-4741-adfb-5e68136b3bab / DEBUG - Message received: {'researcher_id': 'researcher_bc5777a3-5027-465f-8742-3d3aa04a29c9', 'tags': ['#MNIST', '#dataset'], 'command': 'search'}
2022-01-21 09:43:05,134 fedbiomed INFO - log from: node_8dce5575-7403-48f0-91b6-e07e58ae5c47 / DEBUG - Message received: {'researcher_id': 'researcher_bc5777a3-5027-465f-8742-3d3aa04a29c9', 'tags': ['#MNIST', '#dataset'], 'command': 'search'}
01/21/2022 09:43:05:INFO:log from: node_8dce5575-7403-48f0-91b6-e07e58ae5c47 / DEBUG - Message received: {'researcher_id': 'researcher_bc5777a3-5027-465f-8742-3d3aa04a29c9', 'tags': ['#MNIST', '#dataset'], 'command': 'search'}
2022-01-21 09:43:33,430 fedbiomed INFO - log from: node_edb4

2022-01-21 09:43:34,805 fedbiomed INFO - log from: node_8dce5575-7403-48f0-91b6-e07e58ae5c47 / DEBUG - Dataset_path/Users/balelli/data
01/21/2022 09:43:34:INFO:log from: node_8dce5575-7403-48f0-91b6-e07e58ae5c47 / DEBUG - Dataset_path/Users/balelli/data
2022-01-21 09:43:35,465 fedbiomed INFO - log from: node_8dce5575-7403-48f0-91b6-e07e58ae5c47 / INFO - Train Epoch: 1 [0/60000 (0%)]	Loss: 2.312692
01/21/2022 09:43:35:INFO:log from: node_8dce5575-7403-48f0-91b6-e07e58ae5c47 / INFO - Train Epoch: 1 [0/60000 (0%)]	Loss: 2.312692
2022-01-21 09:43:35,587 fedbiomed INFO - log from: node_edb44109-8e5f-4741-adfb-5e68136b3bab / INFO - Train Epoch: 1 [0/60000 (0%)]	Loss: 2.309839
01/21/2022 09:43:35:INFO:log from: node_edb44109-8e5f-4741-adfb-5e68136b3bab / INFO - Train Epoch: 1 [0/60000 (0%)]	Loss: 2.309839
2022-01-21 09:43:40,180 fedbiomed INFO - log from: node_edb44109-8e5f-4741-adfb-5e68136b3bab / INFO - Train Epoch: 1 [480/60000 (1%)]	Loss: 1.304187
01/21/2022 09:43:40:INFO:log from: node_e

2022-01-21 09:44:14,055 fedbiomed INFO - log from: node_edb44109-8e5f-4741-adfb-5e68136b3bab / INFO - Train Epoch: 2 [1440/60000 (2%)]	Loss: 0.224730
01/21/2022 09:44:14:INFO:log from: node_edb44109-8e5f-4741-adfb-5e68136b3bab / INFO - Train Epoch: 2 [1440/60000 (2%)]	Loss: 0.224730
2022-01-21 09:44:14,151 fedbiomed INFO - log from: node_8dce5575-7403-48f0-91b6-e07e58ae5c47 / INFO - Train Epoch: 2 [1440/60000 (2%)]	Loss: 0.175955
01/21/2022 09:44:14:INFO:log from: node_8dce5575-7403-48f0-91b6-e07e58ae5c47 / INFO - Train Epoch: 2 [1440/60000 (2%)]	Loss: 0.175955
2022-01-21 09:44:16,709 fedbiomed INFO - log from: node_8dce5575-7403-48f0-91b6-e07e58ae5c47 / INFO - Train Epoch: 2 [1920/60000 (3%)]	Loss: 0.182656
01/21/2022 09:44:16:INFO:log from: node_8dce5575-7403-48f0-91b6-e07e58ae5c47 / INFO - Train Epoch: 2 [1920/60000 (3%)]	Loss: 0.182656
2022-01-21 09:44:16,783 fedbiomed INFO - log from: node_edb44109-8e5f-4741-adfb-5e68136b3bab / INFO - Train Epoch: 2 [1920/60000 (3%)]	Loss: 0.29589

2022-01-21 09:44:52,976 fedbiomed INFO - log from: node_edb44109-8e5f-4741-adfb-5e68136b3bab / INFO - Train Epoch: 3 [2880/60000 (5%)]	Loss: 0.128928
01/21/2022 09:44:52:INFO:log from: node_edb44109-8e5f-4741-adfb-5e68136b3bab / INFO - Train Epoch: 3 [2880/60000 (5%)]	Loss: 0.128928
2022-01-21 09:44:54,897 fedbiomed INFO - log from: node_8dce5575-7403-48f0-91b6-e07e58ae5c47 / INFO - Train Epoch: 3 [3360/60000 (6%)]	Loss: 0.189708
01/21/2022 09:44:54:INFO:log from: node_8dce5575-7403-48f0-91b6-e07e58ae5c47 / INFO - Train Epoch: 3 [3360/60000 (6%)]	Loss: 0.189708
2022-01-21 09:44:55,267 fedbiomed INFO - log from: node_edb44109-8e5f-4741-adfb-5e68136b3bab / INFO - Train Epoch: 3 [3360/60000 (6%)]	Loss: 0.227840
01/21/2022 09:44:55:INFO:log from: node_edb44109-8e5f-4741-adfb-5e68136b3bab / INFO - Train Epoch: 3 [3360/60000 (6%)]	Loss: 0.227840
2022-01-21 09:44:57,740 fedbiomed INFO - log from: node_8dce5575-7403-48f0-91b6-e07e58ae5c47 / INFO - Train Epoch: 3 [3840/60000 (6%)]	Loss: 0.27145

...

2022-01-21 09:45:50,234 fedbiomed INFO - log from: node_edb44109-8e5f-4741-adfb-5e68136b3bab / INFO - Train Epoch: 2 [0/60000 (0%)]	Loss: 0.093718
01/21/2022 09:45:50:INFO:log from: node_edb44109-8e5f-4741-adfb-5e68136b3bab / INFO - Train Epoch: 2 [0/60000 (0%)]	Loss: 0.093718
2022-01-21 09:45:51,282 fedbiomed INFO - log from: node_8dce5575-7403-48f0-91b6-e07e58ae5c47 / DEBUG - Reached 100 batches for this epoch, ignore remaining data
01/21/2022 09:45:51:INFO:log from: node_8dce5575-7403-48f0-91b6-e07e58ae5c47 / DEBUG - Reached 100 batches for this epoch, ignore remaining data
2022-01-21 09:45:51,970 fedbiomed INFO - log from: node_8dce5575-7403-48f0-91b6-e07e58ae5c47 / INFO - Train Epoch: 2 [0/60000 (0%)]	Loss: 0.109375
01/21/2022 09:45:51:INFO:log from: node_8dce5575-7403-48f0-91b6-e07e58ae5c47 / INFO - Train Epoch: 2 [0/60000 (0%)]	Loss: 0.109375
2022-01-21 09:45:54,265 fedbiomed INFO - log from: node_edb44109-8e5f-4741-adfb-5e68136b3bab / INFO - Train Epoch: 2 [480/60000 (1%)]	Loss

2022-01-21 09:46:31,879 fedbiomed INFO - log from: node_edb44109-8e5f-4741-adfb-5e68136b3bab / INFO - Train Epoch: 3 [1440/60000 (2%)]	Loss: 0.118748
01/21/2022 09:46:31:INFO:log from: node_edb44109-8e5f-4741-adfb-5e68136b3bab / INFO - Train Epoch: 3 [1440/60000 (2%)]	Loss: 0.118748
2022-01-21 09:46:33,646 fedbiomed INFO - log from: node_8dce5575-7403-48f0-91b6-e07e58ae5c47 / INFO - Train Epoch: 3 [1440/60000 (2%)]	Loss: 0.085695
01/21/2022 09:46:33:INFO:log from: node_8dce5575-7403-48f0-91b6-e07e58ae5c47 / INFO - Train Epoch: 3 [1440/60000 (2%)]	Loss: 0.085695
2022-01-21 09:46:34,475 fedbiomed INFO - log from: node_edb44109-8e5f-4741-adfb-5e68136b3bab / INFO - Train Epoch: 3 [1920/60000 (3%)]	Loss: 0.111477
01/21/2022 09:46:34:INFO:log from: node_edb44109-8e5f-4741-adfb-5e68136b3bab / INFO - Train Epoch: 3 [1920/60000 (3%)]	Loss: 0.111477
2022-01-21 09:46:36,465 fedbiomed INFO - log from: node_8dce5575-7403-48f0-91b6-e07e58ae5c47 / INFO - Train Epoch: 3 [1920/60000 (3%)]	Loss: 0.19380

01/21/2022 09:47:02:INFO:log from: node_8dce5575-7403-48f0-91b6-e07e58ae5c47 / DEBUG - Message received: {'researcher_id': 'researcher_bc5777a3-5027-465f-8742-3d3aa04a29c9', 'job_id': 'c9be5f7d-533e-4224-b0df-7a1169c0e542', 'training_args': {'batch_size': 48, 'lr': 0.001, 'epochs': 3, 'dry_run': False, 'batch_maxnum': 100}, 'model_args': {}, 'command': 'train', 'model_url': 'http://localhost:8844/media/uploads/2022/01/21/my_model_80387a84-c8eb-4f87-aaae-d6accbef52ef.py', 'params_url': 'http://localhost:8844/media/uploads/2022/01/21/aggregated_params_e52a3826-ace9-4217-9ee5-f02dba48c9a6.pt', 'model_class': 'MyTrainingPlan', 'training_data': {'node_8dce5575-7403-48f0-91b6-e07e58ae5c47': ['dataset_7adabeb5-d317-4167-a9e5-71b3cd07ab42']}}
2022-01-21 09:47:02,509 fedbiomed INFO - log from: node_8dce5575-7403-48f0-91b6-e07e58ae5c47 / DEBUG - [TASKS QUEUE] Item:{'researcher_id': 'researcher_bc5777a3-5027-465f-8742-3d3aa04a29c9', 'job_id': 'c9be5f7d-533e-4224-b0df-7a1169c0e542', 'params_url': 

2022-01-21 09:47:22,672 fedbiomed INFO - log from: node_edb44109-8e5f-4741-adfb-5e68136b3bab / INFO - Train Epoch: 1 [3840/60000 (6%)]	Loss: 0.098919
01/21/2022 09:47:22:INFO:log from: node_edb44109-8e5f-4741-adfb-5e68136b3bab / INFO - Train Epoch: 1 [3840/60000 (6%)]	Loss: 0.098919
2022-01-21 09:47:23,137 fedbiomed INFO - log from: node_8dce5575-7403-48f0-91b6-e07e58ae5c47 / INFO - Train Epoch: 1 [3840/60000 (6%)]	Loss: 0.230014
01/21/2022 09:47:23:INFO:log from: node_8dce5575-7403-48f0-91b6-e07e58ae5c47 / INFO - Train Epoch: 1 [3840/60000 (6%)]	Loss: 0.230014
2022-01-21 09:47:24,509 fedbiomed INFO - log from: node_edb44109-8e5f-4741-adfb-5e68136b3bab / INFO - Train Epoch: 1 [4320/60000 (7%)]	Loss: 0.044882
01/21/2022 09:47:24:INFO:log from: node_edb44109-8e5f-4741-adfb-5e68136b3bab / INFO - Train Epoch: 1 [4320/60000 (7%)]	Loss: 0.044882
2022-01-21 09:47:25,014 fedbiomed INFO - log from: node_8dce5575-7403-48f0-91b6-e07e58ae5c47 / INFO - Train Epoch: 1 [4320/60000 (7%)]	Loss: 0.24753

2022-01-21 09:47:57,905 fedbiomed INFO - log from: node_8dce5575-7403-48f0-91b6-e07e58ae5c47 / INFO - Train Epoch: 3 [0/60000 (0%)]	Loss: 0.065876
01/21/2022 09:47:57:INFO:log from: node_8dce5575-7403-48f0-91b6-e07e58ae5c47 / INFO - Train Epoch: 3 [0/60000 (0%)]	Loss: 0.065876
2022-01-21 09:47:59,511 fedbiomed INFO - log from: node_edb44109-8e5f-4741-adfb-5e68136b3bab / INFO - Train Epoch: 3 [480/60000 (1%)]	Loss: 0.048119
01/21/2022 09:47:59:INFO:log from: node_edb44109-8e5f-4741-adfb-5e68136b3bab / INFO - Train Epoch: 3 [480/60000 (1%)]	Loss: 0.048119
2022-01-21 09:48:01,388 fedbiomed INFO - log from: node_8dce5575-7403-48f0-91b6-e07e58ae5c47 / INFO - Train Epoch: 3 [480/60000 (1%)]	Loss: 0.056354
01/21/2022 09:48:01:INFO:log from: node_8dce5575-7403-48f0-91b6-e07e58ae5c47 / INFO - Train Epoch: 3 [480/60000 (1%)]	Loss: 0.056354
2022-01-21 09:48:02,227 fedbiomed INFO - log from: node_edb44109-8e5f-4741-adfb-5e68136b3bab / INFO - Train Epoch: 3 [960/60000 (2%)]	Loss: 0.139668
01/21/202